In [19]:
import pandas as pd
from datetime import datetime

from skforecast.ForecasterAutoreg import ForecasterAutoreg
import matplotlib.pyplot as plt
import numpy as np
import shutil
import os
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

def mkdir(d):
    if not os.path.exists(d):
        os.makedirs(d)
        
french_holidays = [
    "2020-07-14",
    "2020-08-15",
    "2020-09-22",
    "2020-11-01",    
    "2020-11-11",
    "2020-12-21",  
    "2020-12-24", 
    "2020-12-25", 
    "2020-12-26", 
    "2020-12-31",     
    
    "2021-01-01",
    "2021-03-20",
    "2021-04-02",
    "2021-04-04",
    "2021-04-05",
    "2021-05-01",
    "2021-05-08",
    "2021-05-13",
    "2021-05-23",
    "2021-05-24",
    "2021-05-30",
    "2021-06-20",
    "2021-06-21",
    "2021-07-14",
    "2021-08-15",    
    "2021-09-22",    
    "2021-11-01",    
    "2021-11-11",     
    "2021-12-21",  
    "2021-12-24", 
    "2021-12-25", 
    "2021-12-26", 
    "2021-12-31", 
]

In [20]:
# Reading input files
train_path = './data/train.csv'
test_path = './data/test.csv'

train = pd.read_csv(train_path, sep=",")
train['date'] = pd.to_datetime(train['date'])
train = train.drop(['Postcode'], axis=1)

# Removing data before corona restrictions
corona_date = datetime.strptime("2020-10-18 00:00:00", '%Y-%m-%d %H:%M:%S')  
train = train[train['date'] > corona_date]

test = pd.read_csv(test_path, sep=",")
test['date'] = pd.to_datetime(test['date'])
test = test.drop(['Postcode'], axis=1)

In [21]:
stations = train['Station'].unique()
dfs = []

for station in stations[:]: # Let's add all the missing datapoints back into the dataframe and fill the known nan's (station, area...)
    single_station_df = train[train['Station'] == station]
    single_station_df = single_station_df.set_index('date').asfreq('15min')
    where_nan = single_station_df.isnull().any(axis=1)
    
    area = single_station_df['area'].value_counts().index.tolist()[0]
    latitude = single_station_df['Latitude'].value_counts().index.tolist()[0]
    longitude = single_station_df['Longitude'].value_counts().index.tolist()[0]

    single_station_df['Station'] = single_station_df['Station'].fillna(station)
    single_station_df['area'] = single_station_df['area'].fillna(area)
    single_station_df['Latitude'] = single_station_df['Latitude'].fillna(latitude)
    single_station_df['Longitude'] = single_station_df['Longitude'].fillna(longitude)

    single_station_df["imputed"] = np.where(where_nan, 1, 0)
    single_station_df['date'] = single_station_df.index
    dfs.append(single_station_df)
    
train = pd.concat(dfs, ignore_index=True).sort_values(by=['date'])

for df in [train, test]:
    df['tod'] = (df.date.dt.hour * 60 + df.date.dt.minute) / 15
    df['dow'] = df.date.dt.dayofweek
    df['month'] = df.date.dt.month
    df['dayofyear'] = df.date.dt.dayofyear
    df['year'] = df.date.dt.year
    
    df['year'] = df['year'].replace([2020], 0)
    df['year'] = df['year'].replace([2021], 1)
    df['year'] = df['year'].replace([2022], 2)
    
    df['tod_sin'] = np.sin(2 * np.pi * df['tod']/96.0)
    df['tod_cos'] = np.cos(2 * np.pi * df['tod']/96.0)

    df['dow_sin'] = np.sin(2 * np.pi * df['dow']/7.0)
    df['dow_cos'] = np.cos(2 * np.pi * df['dow']/7.0)

    df['month_sin'] = np.sin(2 * np.pi * df['month']/12.0)
    df['month_cos'] = np.cos(2 * np.pi * df['month']/12.0)

    df['dayofyear_sin'] = np.sin(2 * np.pi * df['dayofyear']/365.0)
    df['dayofyear_cos'] = np.cos(2 * np.pi * df['dayofyear']/365.0)
    
    df['holiday'] = df['date'].isin(french_holidays).astype(int)

    
test['imputed'] = 0
train

,Station,Available,Charging,Passive,Other,tod,dow,trend,Latitude,Longitude,...,year,tod_sin,tod_cos,dow_sin,dow_cos,month_sin,month_cos,dayofyear_sin,dayofyear_cos,holiday
0,FR*V75*EBELI*1*1,3.0,0.0,0.0,0.0,1.0,6,10300.0,48.85567,2.354089,...,0,0.065403,0.997859,-0.781831,0.623490,-0.866025,0.5,-0.951057,0.309017,0
964143,FR*V75*EBELI*9*1,1.0,0.0,0.0,2.0,1.0,6,10300.0,48.87897,2.293606,...,0,0.065403,0.997859,-0.781831,0.623490,-0.866025,0.5,-0.951057,0.309017,0
119030,FR*V75*EBELI*19*1,3.0,0.0,0.0,0.0,1.0,6,10300.0,48.87741,2.392178,...,0,0.065403,0.997859,-0.781831,0.623490,-0.866025,0.5,-0.951057,0.309017,0
392799,FR*V75*EBELI*4*1,1.0,2.0,0.0,0.0,1.0,6,10300.0,48.87147,2.289894,...,0,0.065403,0.997859,-0.781831,0.623490,-0.866025,0.5,-0.951057,0.309017,0
690374,FR*V75*EBELI*68*1,0.0,0.0,0.0,3.0,1.0,6,10300.0,48.87546,2.360103,...,0,0.065403,0.997859,-0.781831,0.623490,-0.866025,0.5,-0.951057,0.309017,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
904627,FR*V75*EBELI*84*1,3.0,0.0,0.0,0.0,95.0,3,22202.0,48.86377,2.411770,...,1,-0.065403,0.997859,0.433884,-0.900969,0.866025,0.5,0.746972,0.664855,0
380895,FR*V75*EBELI*38*1,2.0,1.0,0.0,0.0,95.0,3,22202.0,48.84793,2.309952,...,1,-0.065403,0.997859,0.433884,-0.900969,0.866025,0.5,0.746972,0.664855,0
702276,FR*V75*EBELI*68*1,0.0,0.0,0.0,3.0,95.0,3,22202.0,48.87546,2.360103,...,1,-0.065403,0.997859,0.433884,-0.900969,0.866025,0.5,0.746972,0.664855,0
595149,FR*V75*EBELI*59*1,0.0,0.0,0.0,3.0,95.0,3,22202.0,48.88328,2.286938,...,1,-0.065403,0.997859,0.433884,-0.900969,0.866025,0.5,0.746972,0.664855,0


In [22]:
targets = ["Available","Charging","Passive","Other"] 

exog_variables_local = ['tod_sin', 'tod_cos',
                  'dow_sin', 'dow_cos', 
                  'month_sin', 'month_cos', 
                  'dayofyear_sin', 'dayofyear_cos', 
                  'year', 'imputed', 'holiday', 
                  'trend']

exog_variables_global = ['Latitude', 'Longitude']

val_local = False # we can set this flag to true to split our dataset into a train and validation set for local evaluation
if val_local: # train val split 
    split_date = datetime.strptime("2021-02-14 00:00:00", '%Y-%m-%d %H:%M:%S')  
    test = train[train['date'] > split_date]    
    train = train[train['date'] <= split_date]
    test[targets[0] + '_GT'] = test[targets[0]]
    test[targets[1] + '_GT'] = test[targets[1]]
    test[targets[2] + '_GT'] = test[targets[2]]
    test[targets[3] + '_GT'] = test[targets[3]]
    test = test.drop(targets, axis=1)

### Impute

In [23]:
# impute train
where_not_null = train.imputed == 0 # add an indicator for values that we impute
where_null = train.imputed == 1

train = train.dropna() # Let's drop na, it works better than any imputation method

'''
# Methods that did not work for imputation

#train[targets] = train[targets].fillna(train.groupby(['date', 'area'])[targets].transform('mean')) # Fill by mean
#train[targets] = train[targets].fillna(train.groupby(['date'])[targets].transform('median')) # Fill by mean

#for station in stations[:]:
#    at_train_station = train['Station'] == station
#    train[at_train_station] = train[at_train_station].fillna(method='bfill')
    #train[at_train_station] = train[at_train_station].interpolate(method='spline', order=5)
    
#train = train.fillna(method='ffill')
#train = train.interpolate(method='nearest')
#train = train.interpolate(method='spline', order=5)
#train = train.interpolate(method='time')
'''

train.isna().sum()

Station          0
Available        0
Charging         0
Passive          0
Other            0
tod              0
dow              0
trend            0
Latitude         0
Longitude        0
area             0
imputed          0
date             0
month            0
dayofyear        0
year             0
tod_sin          0
tod_cos          0
dow_sin          0
dow_cos          0
month_sin        0
month_cos        0
dayofyear_sin    0
dayofyear_cos    0
holiday          0
dtype: int64

### Train & Predict

#### XGBRegressor and ARIMA Model

In [24]:
import xgboost as xgb
from statsmodels.tsa.arima.model import ARIMA

def train_predict_for_station(train_df_single_station, predict_df_single_station):
    prediction_interval_length = len(predict_df_single_station)
    regressor = xgb.XGBRegressor(n_estimators=100)
    forecaster = ForecasterAutoreg(regressor=regressor, lags=20)

    predictions_xgb_regressor = []
    predictions_arima = []
    
    for target in ['Available', 'Charging', 'Passive', 'Other']:
        # train and predict with xgboost
        forecaster.fit(train_df_single_station[target].rolling(10).mean().iloc[10:], 
                       exog=train_df_single_station[exog_variables_local].iloc[10:])
        target_predictions_xgb = forecaster.predict(prediction_interval_length, exog=predict_df_single_station[exog_variables_local])
        predictions_xgb_regressor.append(target_predictions_xgb)
        
        # train and predict with arima
        model = ARIMA(train_df_single_station[target].rolling(10).mean().iloc[10:].reset_index(drop=True), order=(2, 1, 1))
        fitted_arima = model.fit()
        target_predictions_arima = fitted_arima.forecast(prediction_interval_length)
        predictions_arima.append(target_predictions_arima)
        
    return predictions_xgb_regressor, predictions_arima

In [25]:
from tqdm.notebook import tqdm
tqdm.pandas()

print('Training XGBoost Regressor and ARIMA model for all stations (and for each target)')
for i, station in tqdm(enumerate(stations), total=len(stations)):
    at_train_station = train['Station'] == station
    at_test_station = test['Station'] == station
    
    predictions_xgb_regressor, predictions_arima = train_predict_for_station(train[at_train_station], test[at_test_station])
    
    # put preds into dataframe
    index = test[at_test_station].index
    test.loc[at_test_station, 'Available_xgb_regressor'] = pd.Series(predictions_xgb_regressor[0].values, index=index)
    test.loc[at_test_station, 'Charging_xgb_regressor'] = pd.Series(predictions_xgb_regressor[1].values, index=index)
    test.loc[at_test_station, 'Passive_xgb_regressor'] = pd.Series(predictions_xgb_regressor[2].values, index=index)
    test.loc[at_test_station, 'Other_xgb_regressor'] = pd.Series(predictions_xgb_regressor[3].values, index=index)
    
    test.loc[at_test_station, 'Available_arima'] = pd.Series(predictions_arima[0].values, index=index)
    test.loc[at_test_station, 'Charging_arima'] = pd.Series(predictions_arima[1].values, index=index)
    test.loc[at_test_station, 'Passive_arima'] = pd.Series(predictions_arima[2].values, index=index)
    test.loc[at_test_station, 'Other_arima'] = pd.Series(predictions_arima[3].values, index=index)

Training XGBoost Regressor and ARIMA model for all stations (and for each target)


  0%|          | 0/91 [00:00<?, ?it/s]

#### XGB Classifier

In [26]:
# Create Classes from targets
concat = train['Available'].astype(int).astype(str) + train['Charging'].astype(int).astype(str) + train['Passive'].astype(int).astype(str) + train['Other'].astype(int).astype(str)
unique_combos = [c for c in concat.unique() if np.sum([int(digit) for digit in list(c)]) == 3]

assert len(unique_combos) == 20

unique_combos.sort()
unique_combos.reverse()
class_dict = {c: idx for idx, c in enumerate(unique_combos)}
class_dict

def targets_to_class(target_values):
    key = ''.join([str(t) for t in target_values])
    return class_dict[key]
    
for df in [train]:
    df['class'] = df.progress_apply(lambda row: targets_to_class([int(row[target]) for target in targets]), axis=1)

  0%|          | 0/916604 [00:00<?, ?it/s]

/tmp/ipykernel_368/3830655029.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['class'] = df.progress_apply(lambda row: targets_to_class([int(row[target]) for target in targets]), axis=1)


In [27]:
# one-hot-encode stations
station_columns = [f'Station_{station}' for station in stations]

train_station = train['Station']
test_station = test['Station']

train = pd.get_dummies(train, columns=['Station'])
train['Station'] = train_station

test = pd.get_dummies(test, columns=['Station'])
test['Station'] = test_station

# one hot encode are
area_columns = [f'area_{area}' for area in train['area'].unique()]

train_area = train['area']
test_area= test['area']

train = pd.get_dummies(train, columns=['area'])
train['area'] = train_area

test = pd.get_dummies(test, columns=['area'])
test['area'] = test_area

exog_variables = exog_variables_local + exog_variables_global + station_columns + area_columns

train

,Available,Charging,Passive,Other,tod,dow,trend,Latitude,Longitude,imputed,...,Station_FR*V75*EBELI*96*1,Station_FR*V75*EBELI*97*1,Station_FR*V75*EBELI*98*1,Station_FR*V75*EBELI*99*1,Station,area_east,area_north,area_south,area_west,area
0,3.0,0.0,0.0,0.0,1.0,6,10300.0,48.85567,2.354089,0,...,0,0,0,0,FR*V75*EBELI*1*1,0,0,1,0,south
964143,1.0,0.0,0.0,2.0,1.0,6,10300.0,48.87897,2.293606,0,...,0,0,0,0,FR*V75*EBELI*9*1,0,1,0,0,north
119030,3.0,0.0,0.0,0.0,1.0,6,10300.0,48.87741,2.392178,0,...,0,0,0,0,FR*V75*EBELI*19*1,1,0,0,0,east
392799,1.0,2.0,0.0,0.0,1.0,6,10300.0,48.87147,2.289894,0,...,0,0,0,0,FR*V75*EBELI*4*1,0,0,0,1,west
690374,0.0,0.0,0.0,3.0,1.0,6,10300.0,48.87546,2.360103,0,...,0,0,0,0,FR*V75*EBELI*68*1,1,0,0,0,east
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
904627,3.0,0.0,0.0,0.0,95.0,3,22202.0,48.86377,2.411770,0,...,0,0,0,0,FR*V75*EBELI*84*1,1,0,0,0,east
380895,2.0,1.0,0.0,0.0,95.0,3,22202.0,48.84793,2.309952,0,...,0,0,0,0,FR*V75*EBELI*38*1,0,0,0,1,west
702276,0.0,0.0,0.0,3.0,95.0,3,22202.0,48.87546,2.360103,0,...,0,0,0,0,FR*V75*EBELI*68*1,1,0,0,0,east
595149,0.0,0.0,0.0,3.0,95.0,3,22202.0,48.88328,2.286938,0,...,0,0,0,0,FR*V75*EBELI*59*1,0,1,0,0,north


In [29]:
classifier = xgb.XGBClassifier(n_estimators=300) 
classifier.fit(train[exog_variables], train['class'], eval_set=[(train[exog_variables], train['class'])])
predictions = classifier.predict(test[exog_variables])

test['predicted_class'] = predictions

def class_to_target(clazz: int, target: str) -> int:
    target_str = list(class_dict.keys())[list(class_dict.values()).index(clazz)]
    return int(target_str[targets.index(target)])

for t in targets:
    test[t + "_xgb_classifier"] = test['predicted_class'].apply(lambda pred: class_to_target(pred, target=t))

[0]	validation_0-mlogloss:2.27015
[1]	validation_0-mlogloss:2.02239
[2]	validation_0-mlogloss:1.86486
[3]	validation_0-mlogloss:1.74043
[4]	validation_0-mlogloss:1.64735
[5]	validation_0-mlogloss:1.57706
[6]	validation_0-mlogloss:1.51574
[7]	validation_0-mlogloss:1.47121
[8]	validation_0-mlogloss:1.43684
[9]	validation_0-mlogloss:1.40266
[10]	validation_0-mlogloss:1.37628
[11]	validation_0-mlogloss:1.35000
[12]	validation_0-mlogloss:1.32910
[13]	validation_0-mlogloss:1.31195
[14]	validation_0-mlogloss:1.29856
[15]	validation_0-mlogloss:1.28411
[16]	validation_0-mlogloss:1.27013
[17]	validation_0-mlogloss:1.26018
[18]	validation_0-mlogloss:1.24947
[19]	validation_0-mlogloss:1.24125
[20]	validation_0-mlogloss:1.23091
[21]	validation_0-mlogloss:1.22219
[22]	validation_0-mlogloss:1.21443
[23]	validation_0-mlogloss:1.20710
[24]	validation_0-mlogloss:1.20112
[25]	validation_0-mlogloss:1.19511
[26]	validation_0-mlogloss:1.18955
[27]	validation_0-mlogloss:1.18463
[28]	validation_0-mlogloss:1.1

In [28]:
test_copy = test.copy()

### Ensembling the predictions and scaling them

In [30]:
xgbr_preds = ['Available_xgb_regressor', 'Charging_xgb_regressor', 'Passive_xgb_regressor', 'Other_xgb_regressor']
arima_preds = ['Available_arima', 'Charging_arima', 'Passive_arima', 'Other_arima']
xgbc_preds = ['Available_xgb_classifier', 'Charging_xgb_classifier', 'Passive_xgb_classifier', 'Other_xgb_classifier']

def scale_nums_to_sum(nums, station, value=3):
    if np.sum(nums) == 0:
        return [0, 0, 0, 0]
    return [num / np.sum(nums) * value for num in nums]


def round_and_rescale(df, cols):
    for col in cols: 
        df.loc[test[col] < 0, col] = 0
        df.loc[test[col] > 3, col] = 3
    df[cols] = df[cols].apply(np.round)
    
    k = df.apply(lambda row : scale_nums_to_sum([row[cols[0]],
                                                 row[cols[1]], 
                                                 row[cols[2]],
                                                 row[cols[3]]],
                                                 row["Station"]), axis = 1)
    k = np.array(k.to_list())

    df[cols[0]] = k[:, 0]
    df[cols[1]] = k[:, 1]
    df[cols[2]] = k[:, 2]
    df[cols[3]] = k[:, 3]
    
    return df


def rescale_and_round(df, cols):
    for col in cols: 
        df.loc[test[col] < 0, col] = 0
        df.loc[test[col] > 3, col] = 3
    
    
    k = df.apply(lambda row : scale_nums_to_sum([row[cols[0]],
                                                 row[cols[1]], 
                                                 row[cols[2]],
                                                 row[cols[3]]],
                                                 row["Station"]), axis = 1)
    k = np.array(k.to_list())

    df[cols[0]] = k[:, 0]
    df[cols[1]] = k[:, 1]
    df[cols[2]] = k[:, 2]
    df[cols[3]] = k[:, 3]
    
    df[cols] = df[cols].apply(np.round)
    
    return df

test = rescale_and_round(test, xgbr_preds)
test = round_and_rescale(test, arima_preds)

for xgbr_pred, arima_pred, xgbc_pred, target in zip(xgbr_preds, arima_preds, xgbc_preds, targets):    
    test[target] = test[arima_pred] * 0.4 + test[xgbr_pred] * 0.35 + test[xgbc_pred] * 0.25
    
test = rescale_and_round(test, targets)

In [52]:
test[targets]

,Available,Charging,Passive,Other
0,3.0,0.0,0.0,0.0
1,0.0,0.0,0.0,3.0
2,0.0,0.0,0.0,3.0
3,3.0,0.0,0.0,0.0
4,3.0,0.0,0.0,0.0
...,...,...,...,...
165979,0.0,0.0,0.0,3.0
165980,2.0,1.0,0.0,0.0
165981,2.0,0.0,0.0,0.0
165982,2.0,0.0,1.0,0.0


In [54]:
from sklearn.metrics import mean_absolute_error
import os

if val_local: # Let's check how we did on the validation set
    station_maes = []
    for station in stations:
        at_station = test['Station'] == station
        target_maes = []

        for target in targets:
            error = mean_absolute_error(test.dropna()[at_station][target], test.dropna()[at_station][target + '_GT'])
            target_maes.append(error)

        station_maes.append(np.sum(target_maes))

    print("All stations mea:", np.mean(station_maes))
    plt.figure(figsize=(15,20))
    plt.barh(stations, station_maes)
    plt.show()
    
else: # Let's make a submission!
    pred_area = test.groupby(['date', 'area']).agg({
        'Available': 'sum',
        'Charging': 'sum',
        'Passive': 'sum',
        'Other': 'sum'}).reset_index()

    pred_global = test.groupby('date').agg({
        'Available': 'sum',
        'Charging': 'sum',
        'Passive': 'sum',
        'Other': 'sum'}).reset_index()
    
    mkdir("result")

    test[["date","area","Station","Available","Charging","Passive","Other"]].reset_index(drop=True).to_csv("./result/station.csv", index=False)
    test[["date","area","Available","Charging","Passive","Other"]].reset_index(drop=True).to_csv("./result/area.csv", index=False)
    test[["date","Available","Charging","Passive","Other"]].reset_index(drop=True).to_csv("./result/global.csv", index=False)
    shutil.make_archive("./result",'zip', "./result")